This notebook tries to search for optimal fixed policies (e.g. constant mortality) that maximize the objective (i.e. expected net reward). Here I try [scikit-optimize](https://scikit-optimize.github.io/stable/index.html) routines which are designed for noisy functions and compare to a brute-force parallel grid search.  

In [47]:
%pip install -e ..
# %pip install scikit-optimize

Obtaining file:///home/rstudio/rl4fisheries
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for rl4fisheries (pyproject.toml) ... done
  Created wheel for rl4fisheries: filename=rl4fisheries-1.0.0-0.editable-py3-none-any.whl size=2176 sha256=aebb65ca4f07d99d588c7fb0de18b7cdbb9aff0bb29ab44fe3dd315eb92caf22
  Stored in directory: /tmp/pip-ephem-wheel-cache-t5m_i4it/wheels/d3/ce/fe/d5af67bb4edf309f6a59d59140b2b78d5a336b2ad4b93a1fb4
Successfully built rl4fisheries
  Attempting uninstall: rl4fisheries
    Found existing installation: rl4fisheries 1.0.0
    Uninstalling rl4fisheries-1.0.0:
      Successfully uninstalled rl4fisheries-1.0.0
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [45]:
from rl4caribou import Caribou
from skopt import gp_minimize, gbrt_minimize
import polars as pl
import numpy as np
from plotnine import ggplot, aes, geom_point, geom_ribbon


Here is an example of a simple fixed action policy.  It will apply a fixed hunting effort (potentially zero) each year to Moose, and another fixed effort to Wolves.  

In [30]:
class fixed_effort:
    def __init__(self, action):
        self.effort = np.array(action, dtype=np.float32)

    def predict(self, observation, **kwargs):
        action = self.effort * 2 - 1
        return action, {}

pacifist = fixed_effort([0., 0.])

In [37]:
env = Caribou()
obs = env.reset()
action, _ = pacifist.predict(obs)
env.step(action)

(array([-0.4943695 , -0.48672426, -0.8089408 ], dtype=float32),
 0.250531405210495,
 False,
 False,
 {})

## Fixed policy evaluation helpers

This function simulates the dynamics under any given manager.  Each timestep, the manager gets an observation of the population (Caribou, Moose, Wolves), and decides ("predicts") what harvest action to take on wolves and moose to maximize the overall net utility over the full simulation.

A helper utility runs this simulation 10 times and returns the mean and summary statistics.  

In [54]:
def gen_ep_rew(manager, env):
    episode_reward = 0.0
    observation, _ = env.reset()
    for t in range(env.Tmax):
        action, _ = manager.predict(observation)
        observation, reward, terminated, done, info = env.step(action)
        episode_reward += reward
        if terminated or done:
            break
    return episode_reward

def gather_stats(manager, env, N=10):
    results = [gen_ep_rew(manager, env) for _ in range(N)]
    y = np.mean(results)
    sigma = np.std(results)
    ymin = y - sigma
    ymax = y + sigma
    return y, ymin, ymax 

In [40]:
gen_ep_rew(pacifist, env)
gather_stats(pacifist, env)

(23.70720968544483, 5.766838293769911, 41.64758107711975)

## Determine optimal mortality policy

Use Bayesian optimization techniques for nonlinear and stochastic functions from Scikit-Optimize (e.g. Gaussian Process estimation) to estimate the optimal fixed mortality policy for both wolves and moose: (err, maybe this can be done analytically too).  Note we define the function to be minimized, `g(x)` as a function of the actions, `x`. Note we report the _negative_ mean reward since the optimizer tries to _minimize_ the value.  

In [49]:
def g(x):
    manager = fixed_effort(x)
    out = gather_stats(manager, env)
    return - out[0]

In [50]:
%%time
res = gp_minimize(g, [(0.0, 0.3), (0, 0.3)], n_calls = 300)
res.fun, res.x

CPU times: user 2min 42s, sys: 9min 48s, total: 12min 31s
Wall time: 1min 40s


(-192.28646437703884, [0.17790704682764627, 0.061024282615602])

In [53]:
%%time
res = gbrt_minimize(g, [(0.0, 0.3), (0, 0.3)], n_calls = 300)
res.fun, res.x

CPU times: user 3min 41s, sys: 644 ms, total: 3min 42s
Wall time: 3min 40s


(-183.45138428616946, [0.1625976286665279, 0.05916838814404951])